In [1]:
import findspark
import os
import gc
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
findspark.init(SPARK_HOME)

import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
import numpy as np
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import size, array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
import geofeather
from pyspark.storagelevel import StorageLevel

from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [2]:
tin_file = input("Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file:")

# get the directory, basename of the input file
print("\n********************")
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2):") or "2"

if directory_type == '0':
    directory = 'hdfs_data'
elif directory_type == '1':
    directory = 'Tri_data'
else:
    directory = 'Tetra_data'
    
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)

print("\n********************")
print("This is a TIN file in \"%s\" format" % tin_extension)

filtra = 'yes'

# allocate the number of executors, the number of cores per executor, and the amount of memory per executor
Num_executor = '64'
Num_core_per_executor = '5'
Memory_executor = '64g'
MemoryOverhead_executor = '8g'

# allocate the number of cores for the driver node
Num_core_per_driver = '5'
Memory_driver = '64g'
MemoryOverhead_driver = '32g'

# the default number of shuffle partitions
Num_shuffle_partitions = input("spark.sql.shuffle.partitions:")


********************
tin_directory:  /home/qiany/yuehui/pyspark/Tetra_mesh/data
tin_basename:  Lander_big.ts
tin_filename:  Lander_big
tin_extension:  .ts

********************
This is a TIN file in ".ts" format


In [3]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

Topo = input("What kind of connectivity relation do you want to retrieve?")

spark_app_name = "TopoRela_CoBoundary_TetraMesh_" + tin_filename + '_' + date_name + '_' + hour_name + '_' + Topo
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '400') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.sql.objectHashAggregate.sortBased.fallbackThreshold", "-1") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config("spark.eventLog.logStageExecutorMetrics", "true") \
.getOrCreate()

What kind of connectivity relation do you want to retrieve? EF_M4_show


spark_app_name: TopoRela_CoBoundary_TetraMesh_Lander_big_06142024_1558_EF_M4_show


### read input files

In [4]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tetra_order(hdfs_tetra_origin):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a tetrahedra file
    tin_filename: a string denoting the file name of a tetrahedra extension, e.g., 827_monviso
    '''
        
    schema_tetra_origin = StructType([ \
        StructField("tetra_order",IntegerType(),True), \
        StructField("r1",IntegerType(),True), \
        StructField("r2",IntegerType(),True), \
        StructField("r3",IntegerType(),True), \
        StructField("r4",IntegerType(),True), \
        StructField("r1_ele",FloatType(),True), \
        StructField("r2_ele",FloatType(),True), \
        StructField("r3_ele",FloatType(),True), \
        StructField("r4_ele",FloatType(),True) \
      ])

    df_tetra_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tetra_origin)\
          .load(hdfs_tetra_origin)
        
    return df_tetra_origin


# read tetrahedra
hdfs_tetra_origin = directory + "/" + tin_filename + '_filtra_tetra_sort.csv'

df_tetra_order = read_tetra_order(hdfs_tetra_origin)
df_tetra_order.printSchema()

Number of partitions for df_tetra_origin: 4959
root
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- r4_ele: float (nullable = true)



# obtain VT relation

### Method 1: pure global

In [5]:
# function to get VT directly from DF_T
def get_VT(df_tetra_order):
    df_tetra_order = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False))
    df_VT_init_1 = df_tetra_order.select("r1","tetra")
    df_VT_init_2 = df_tetra_order.select("r2","tetra")
    df_VT_init_3 = df_tetra_order.select("r3","tetra")
    df_VT_init_4 = df_tetra_order.select("r4","tetra")
    
    df_VT_union12 = df_VT_init_1.union(df_VT_init_2)
    df_VT_union123 = df_VT_union12.union(df_VT_init_3)
    df_VT_union1234 = df_VT_union123.union(df_VT_init_4)
    
    df_VT = df_VT_union1234.groupBy('r1').agg(collect_list('tetra').alias('VT'))
    df_VT = df_VT.withColumnRenamed('r1', 'Ver')
    
    return df_VT

In [6]:
df_VT = get_VT(df_tetra_order)

df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [7]:
t0 = time.time()

num_row = df_VT.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 498.6207056045532
number of rows: 1792989718


In [8]:
t0 = time.time()

file_VT = directory + '/' + tin_filename + '_VT.parquet'

df_VT.write.parquet(file_VT)

t1 = time.time()
print("time cost:", t1-t0)

time cost: 1447.6604726314545


# obtain VF relation

### Method 1: pure global

In [5]:
# function to get VF from DF_T
def get_VF_init(df_tetra_order):
    df_VF_1 = df_tetra_order.withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).drop('r2', 'r3', 'r4')
    df_VF_2 = df_tetra_order.withColumn("f1", sort_array(F.array("r2", "r1", "r3"), False)).withColumn("f2", sort_array(F.array("r2", "r1", "r4"), False)).withColumn("f3", sort_array(F.array("r2", "r3", "r4"), False)).drop('r1', 'r3', 'r4')
    df_VF_3 = df_tetra_order.withColumn("f1", sort_array(F.array("r3", "r1", "r2"), False)).withColumn("f2", sort_array(F.array("r3", "r1", "r4"), False)).withColumn("f3", sort_array(F.array("r3", "r2", "r4"), False)).drop('r1', 'r2', 'r4')
    df_VF_4 = df_tetra_order.withColumn("f1", sort_array(F.array("r4", "r1", "r2"), False)).withColumn("f2", sort_array(F.array("r4", "r1", "r3"), False)).withColumn("f3", sort_array(F.array("r4", "r2", "r3"), False)).drop('r1', 'r2', 'r3')
    
    df_VF_union12 = df_VF_1.union(df_VF_2)
    df_VF_union123 = df_VF_union12.union(df_VF_3)
    df_VF_union1234 = df_VF_union123.union(df_VF_4)
    
    df_VF_init = df_VF_union1234.groupBy('r1').agg(collect_set('f1').alias('multi_f1'), collect_set('f2').alias('multi_f2'), collect_set('f3').alias('multi_f3'))
    return df_VF_init

df_VF_init = get_VF_init(df_tetra_order)

# df_VE_init.cache()

df_VF_init.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_f1: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_f2: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_f3: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [6]:
# obtain VF relation
def get_VF(multi_f1, multi_f2, multi_f3):
# get_VF is used to obtain a complete VF relation from the partial VF relations
# multi_f1: partial VF relation
# multi_f2: partial VF relation
# multi_f3: partial VF relation

    faces = set()
    for f in multi_f1:
        faces.add(tuple(f))
        
    for f in multi_f2:
        faces.add(tuple(f))
        
    for f in multi_f3:
        faces.add(tuple(f))
    
    faces_list = sorted(faces) # save more time when using list(faces)
    
    return faces_list

get_VF_udf = udf(get_VF, ArrayType(ArrayType(IntegerType())))

df_VF = df_VF_init.withColumn("VF", get_VF_udf(df_VF_init.multi_f1, df_VF_init.multi_f2, df_VF_init.multi_f3)).drop('multi_f1', 'multi_f2', 'multi_f3')
df_VF = df_VF.withColumnRenamed('r1', 'Ver')

df_VF.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [7]:
t0 = time.time()

num_row = df_VF.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 565.2756164073944
number of rows: 1792989718


# obtain VE relation

### Method 1: pure global

In [5]:
# function to get VE from DF_T
def get_VE_init(df_tetra_order):
    df_VE_1 = df_tetra_order.withColumn("e1", sort_array(F.array("r1", "r2"), False)).withColumn("e2", sort_array(F.array("r1", "r3"), False)).withColumn("e3", sort_array(F.array("r1", "r4"), False)).drop('r2', 'r3', 'r4')
    df_VE_2 = df_tetra_order.withColumn("e1", sort_array(F.array("r2", "r1"), False)).withColumn("e2", sort_array(F.array("r2", "r3"), False)).withColumn("e3", sort_array(F.array("r2", "r4"), False)).drop('r1', 'r3', 'r4')
    df_VE_3 = df_tetra_order.withColumn("e1", sort_array(F.array("r3", "r1"), False)).withColumn("e2", sort_array(F.array("r3", "r2"), False)).withColumn("e3", sort_array(F.array("r3", "r4"), False)).drop('r1', 'r2', 'r4')
    df_VE_4 = df_tetra_order.withColumn("e1", sort_array(F.array("r4", "r1"), False)).withColumn("e2", sort_array(F.array("r4", "r2"), False)).withColumn("e3", sort_array(F.array("r4", "r3"), False)).drop('r1', 'r2', 'r3')
    
    df_VE_union1234 = df_VE_1.union(df_VE_2).union(df_VE_3).union(df_VE_4)
    
    df_VE_init = df_VE_union1234.groupBy('r1').agg(collect_set('e1').alias('multi_e1'), collect_set('e2').alias('multi_e2'), collect_set('e3').alias('multi_e3'))
    return df_VE_init

df_VE_init = get_VE_init(df_tetra_order)

# df_VE_init.cache()

df_VE_init.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_e1: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_e2: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_e3: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [6]:
# obtain VE relation
def get_VE(multi_e1, multi_e2, multi_e3):
# get_VE is used to obtain the partial VE relation
# multi_e1: partial VE relation
# multi_e2: partial VE relation
    edges = set()
    for e in multi_e1:
        edges.add(tuple(e))
        
    for e in multi_e2:
        edges.add(tuple(e))
        
    for e in multi_e3:
        edges.add(tuple(e))
    
    edges_list = sorted(edges) # sorted(pt_set, reverse=False), False in ascending order while True in descending order
    
    return edges_list

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_VE_udf = udf(get_VE, ArrayType(ArrayType(IntegerType())))

df_VE = df_VE_init.withColumn("VE", get_VE_udf(df_VE_init.multi_e1, df_VE_init.multi_e2, df_VE_init.multi_e3)).drop('multi_e1', 'multi_e2', 'multi_e3')
df_VE = df_VE.withColumnRenamed('r1', 'Ver')

df_VE.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [7]:
t0 = time.time()

num_row = df_VE.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 501.4733638763428
number of rows: 1792989718


# obtain EF relation

### Method 1: pure global

##### first get EV

In [7]:
# function to get TE relation
def get_TE_from_T(df_tetra_order):
    df_TE = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("e1", sort_array(F.array("r1", "r2",), False)).withColumn("e2", sort_array(F.array("r1", "r3",), False)).withColumn("e3", sort_array(F.array("r1", "r4",), False)).withColumn("e4", sort_array(F.array("r2", "r3",), False)).withColumn("e5", sort_array(F.array("r2", "r4",), False)).withColumn("e6", sort_array(F.array("r3", "r4",), False))
    df_TE = df_TE.select('tetra', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6')
    
    return df_TE

In [8]:
# get edges from DF_T

df_TE = get_TE_from_T(df_tetra_order)

df_E1 = df_TE.select('e1')
df_E2 = df_TE.select('e2')
df_E3 = df_TE.select('e3')
df_E4 = df_TE.select('e4')
df_E5 = df_TE.select('e5')
df_E6 = df_TE.select('e6')

df_Edges = df_E1.union(df_E2).union(df_E3).union(df_E4).union(df_E5).union(df_E6)
df_Edges = df_Edges.withColumnRenamed('e1', 'edge')
df_Edges = df_Edges.distinct()
df_Edges.printSchema()

root
 |-- edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [9]:
df_EV = df_Edges.withColumn('r1', df_Edges.edge[0]).withColumn('r2', df_Edges.edge[1])
df_EV.printSchema()

root
 |-- edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)



##### then get VF

In [10]:
# function to get VF from DF_T
def get_VF_init(df_tetra_order):
    df_VF_1 = df_tetra_order.withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).drop('r2', 'r3', 'r4')
    df_VF_2 = df_tetra_order.withColumn("f1", sort_array(F.array("r2", "r1", "r3"), False)).withColumn("f2", sort_array(F.array("r2", "r1", "r4"), False)).withColumn("f3", sort_array(F.array("r2", "r3", "r4"), False)).drop('r1', 'r3', 'r4')
    df_VF_3 = df_tetra_order.withColumn("f1", sort_array(F.array("r3", "r1", "r2"), False)).withColumn("f2", sort_array(F.array("r3", "r1", "r4"), False)).withColumn("f3", sort_array(F.array("r3", "r2", "r4"), False)).drop('r1', 'r2', 'r4')
    df_VF_4 = df_tetra_order.withColumn("f1", sort_array(F.array("r4", "r1", "r2"), False)).withColumn("f2", sort_array(F.array("r4", "r1", "r3"), False)).withColumn("f3", sort_array(F.array("r4", "r2", "r3"), False)).drop('r1', 'r2', 'r3')
    
    df_VF_union12 = df_VF_1.union(df_VF_2)
    df_VF_union123 = df_VF_union12.union(df_VF_3)
    df_VF_union1234 = df_VF_union123.union(df_VF_4)
    
    df_VF_init = df_VF_union1234.groupBy('r1').agg(collect_set('f1').alias('multi_f1'), collect_set('f2').alias('multi_f2'), collect_set('f3').alias('multi_f3'))
    return df_VF_init

df_VF_init = get_VF_init(df_tetra_order)

# df_VE_init.cache()

df_VF_init.printSchema()

root
 |-- r1: integer (nullable = true)
 |-- multi_f1: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_f2: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- multi_f3: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [11]:
# obtain VF relation
def get_VF(multi_f1, multi_f2, multi_f3):
# get_VF is used to obtain a complete VF relation from the partial VF relations
# multi_f1: partial VF relation
# multi_f2: partial VF relation
# multi_f3: partial VF relation

    faces = set()
    for f in multi_f1:
        faces.add(tuple(f))
        
    for f in multi_f2:
        faces.add(tuple(f))
        
    for f in multi_f3:
        faces.add(tuple(f))
    
    faces_list = sorted(faces) # save more time when using list(faces)
    
    return faces_list

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_VF_udf = udf(get_VF, ArrayType(ArrayType(IntegerType())))

df_VF = df_VF_init.withColumn("VF", get_VF_udf(df_VF_init.multi_f1, df_VF_init.multi_f2, df_VF_init.multi_f3)).drop('multi_f1', 'multi_f2', 'multi_f3')
df_VF = df_VF.withColumnRenamed('r1', 'Ver')

df_VF.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



##### join EV and VF

In [12]:
df_EF_init_1 = df_EV.join(df_VF, df_EV.r1==df_VF.Ver).select("edge", col("VF").alias("VF_1"), "r2")
df_EF_init_2 = df_EF_init_1.join(df_VF, df_EF_init_1.r2==df_VF.Ver).select("edge", "VF_1", col("VF").alias("VF_2"))

In [13]:
def get_EF(edge, VF_1, VF_2):
    EF = set()
    edge_set = set(edge)
    for face in VF_1:
        face_set = set(face)
        if edge_set.issubset(face_set):
            ET.add(tuple(face))
            
    for face in VF_2:
        face_set = set(face)
        if edge_set.issubset(face_set):
            ET.add(tuple(face))
            
    EF_list = list(EF)
    
    return EF_list

get_EF_udf = udf(get_EF, ArrayType(ArrayType(IntegerType())))

In [14]:
df_EF = df_EF_init_2.withColumn("EF", get_EF_udf(df_EF_init_2.edge, df_EF_init_2.VF_1, df_EF_init_2.VF_2)).select("edge", "EF")
df_EF.printSchema()

root
 |-- edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- EF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [15]:
t0 = time.time()

num_row = df_EF.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 1628.194329738617
number of rows: 11511582639


# obtain ET relation

### Method 1: pure global

##### first get EV

In [5]:
# function to get TE relation
def get_TE_from_T(df_tetra_order):
    df_TE = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("e1", sort_array(F.array("r1", "r2",), False)).withColumn("e2", sort_array(F.array("r1", "r3",), False)).withColumn("e3", sort_array(F.array("r1", "r4",), False)).withColumn("e4", sort_array(F.array("r2", "r3",), False)).withColumn("e5", sort_array(F.array("r2", "r4",), False)).withColumn("e6", sort_array(F.array("r3", "r4",), False))
    df_TE = df_TE.select('tetra', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6')
    
    return df_TE

In [6]:
# get edges from DF_T

df_TE = get_TE_from_T(df_tetra_order)

df_E1 = df_TE.select('e1')
df_E2 = df_TE.select('e2')
df_E3 = df_TE.select('e3')
df_E4 = df_TE.select('e4')
df_E5 = df_TE.select('e5')
df_E6 = df_TE.select('e6')

df_Edges = df_E1.union(df_E2).union(df_E3).union(df_E4).union(df_E5).union(df_E6)
df_Edges = df_Edges.withColumnRenamed('e1', 'edge')
df_Edges = df_Edges.distinct()
df_Edges.printSchema()

root
 |-- edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [7]:
df_EV = df_Edges.withColumn('r1', df_Edges.edge[0]).withColumn('r2', df_Edges.edge[1])
df_EV.printSchema()

root
 |-- edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)



##### then get VT

In [8]:
# function to get VT directly from DF_T
def get_VT(df_tetra_order):
    df_tetra_order = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False))
    df_VT_init_1 = df_tetra_order.select("r1","tetra")
    df_VT_init_2 = df_tetra_order.select("r2","tetra")
    df_VT_init_3 = df_tetra_order.select("r3","tetra")
    df_VT_init_4 = df_tetra_order.select("r4","tetra")
    
    df_VT_union12 = df_VT_init_1.union(df_VT_init_2)
    df_VT_union123 = df_VT_union12.union(df_VT_init_3)
    df_VT_union1234 = df_VT_union123.union(df_VT_init_4)
    
    df_VT = df_VT_union1234.groupBy('r1').agg(collect_list('tetra').alias('VT'))
    df_VT = df_VT.withColumnRenamed('r1', 'Ver')
    
    return df_VT

In [9]:
df_VT = get_VT(df_tetra_order)

df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



##### join df_EV and df_VT

In [10]:
df_ET_init_1 = df_EV.join(df_VT, df_EV.r1==df_VT.Ver).select("edge", col("VT").alias("VT_1"), "r2")
df_ET_init_2 = df_ET_init_1.join(df_VT, df_ET_init_1.r2==df_VT.Ver).select("edge", "VT_1", col("VT").alias("VT_2"))

In [11]:
def get_ET(edge, VT_1, VT_2):
    ET = set()
    edge_set = set(edge)
    for tetra in VT_1:
        tetra_set = set(tetra)
        if edge_set.issubset(tetra_set):
            ET.add(tuple(tetra))
            
    for tetra in VT_2:
        tetra_set = set(tetra)
        if edge_set.issubset(tetra_set):
            ET.add(tuple(tetra))
            
    ET_list = list(ET)
    
    return ET_list

get_ET_udf = udf(get_ET, ArrayType(ArrayType(IntegerType())))

In [12]:
df_ET = df_ET_init_2.withColumn("ET", get_ET_udf(df_ET_init_2.edge, df_ET_init_2.VT_1, df_ET_init_2.VT_2)).select("edge", "ET")
df_ET.printSchema()

root
 |-- edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- ET: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [13]:
t0 = time.time()

num_row = df_ET.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 1649.0040829181671
number of rows: 11511582639


# obtain FT relation

### Method 1: pure global

##### first get FV

In [7]:
# function to get TF relation
def get_TF_from_T(df_tetra_order):
    df_TF = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).withColumn("f4", sort_array(F.array("r2", "r3", "r4"), False))
    df_TF = df_TF.select('tetra', 'f1', 'f2', 'f3', 'f4')
    
    return df_TF

In [8]:
# get faces from DF_T

df_TF = get_TF_from_T(df_tetra_order)

df_F1 = df_TF.select('f1')
df_F2 = df_TF.select('f2')
df_F3 = df_TF.select('f3')
df_F4 = df_TF.select('f4')

df_Faces = df_F1.union(df_F2).union(df_F3).union(df_F4)
df_Faces = df_Faces.withColumnRenamed('f1', 'face')
df_Faces = df_Faces.distinct()
df_Faces.printSchema()

root
 |-- face: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [9]:
df_FV = df_Faces.withColumn('r1', df_Faces.face[0]).withColumn('r2', df_Faces.face[1]).withColumn('r3', df_Faces.face[2])
df_FV.printSchema()

root
 |-- face: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)



##### then get VT

In [10]:
# function to get VT directly from DF_T
def get_VT(df_tetra_order):
    df_tetra_order = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False))
    df_VT_init_1 = df_tetra_order.select("r1","tetra")
    df_VT_init_2 = df_tetra_order.select("r2","tetra")
    df_VT_init_3 = df_tetra_order.select("r3","tetra")
    df_VT_init_4 = df_tetra_order.select("r4","tetra")
    
    df_VT_union12 = df_VT_init_1.union(df_VT_init_2)
    df_VT_union123 = df_VT_union12.union(df_VT_init_3)
    df_VT_union1234 = df_VT_union123.union(df_VT_init_4)
    
    df_VT = df_VT_union1234.groupBy('r1').agg(collect_list('tetra').alias('VT'))
    df_VT = df_VT.withColumnRenamed('r1', 'Ver')
    
    return df_VT

In [11]:
df_VT = get_VT(df_tetra_order)

df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



##### join df_FV and df_VT

In [12]:
df_FT_init_1 = df_FV.join(df_VT, df_FV.r1==df_VT.Ver).select("face", col("VT").alias("VT_1"), "r2", "r3")
df_FT_init_2 = df_FT_init_1.join(df_VT, df_FT_init_1.r2==df_VT.Ver).select("face", "VT_1", col("VT").alias("VT_2"), "r3")
df_FT_init_3 = df_FT_init_2.join(df_VT, df_FT_init_2.r3==df_VT.Ver).select("face", "VT_1", "VT_2", col("VT").alias("VT_3"))

In [13]:
def get_FT(face, VT_1, VT_2, VT_3):
    FT = set()
    face_set = set(face)
    for tetra in VT_1:
        tetra_set = set(tetra)
        if face_set.issubset(tetra_set):
            FT.add(tuple(tetra))
            
    for tetra in VT_2:
        tetra_set = set(tetra)
        if face_set.issubset(tetra_set):
            FT.add(tuple(tetra))
            
    for tetra in VT_3:
        tetra_set = set(tetra)
        if face_set.issubset(tetra_set):
            FT.add(tuple(tetra))
            
    FT_list = list(FT)
    
    return FT_list

get_FT_udf = udf(get_FT, ArrayType(ArrayType(IntegerType())))

In [14]:
df_FT = df_FT_init_3.withColumn("FT", get_FT_udf(df_FT_init_3.face, df_FT_init_3.VT_1, df_FT_init_3.VT_2, df_FT_init_3.VT_3)).select("face", "FT")
df_FT.printSchema()

root
 |-- face: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- FT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [15]:
t0 = time.time()

num_row = df_FT.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 1778.6693172454834
number of rows: 19434269798
